<a href="https://colab.research.google.com/github/so5/YOLO-seminar/blob/main/YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このnotebookは [公式webサイト](https://pjreddie.com/darknet/yolo/) の "Detection Using A Pre-Trained Model" セクションに書かれている内容を順にgoogle colab上で実行するものです。


---



#事前準備

これ以降の手順を実行するには、次のツール、ライブラリが必要になります。
- git
- GNU make
- Cコンパイラ(gcc等)
- OpenCV
- wget

google colabの場合、`wget`以外はインストール済なので`wget`のみをインストールします。

In [ ]:
!apt update && apt install wget

# darknetのビルド
darknetはC言語で書かれたプログラムなので、pythonなどのプログラムとは違い
ソースコードから実行ファイルをビルドする必要があります。

In [ ]:
!git clone https://github.com/pjreddie/darknet 

In [ ]:
cd darknet

In [ ]:
!make OPENCV=1 all

#成果物の確認

正常にビルドが終了していれば、`darknet` という実行ファイルが生成されています。
また、`cfg` ディレクトリの中には複数の機械学習モデルの設定ファイルが入っているので確認してみましょう。


In [ ]:
ls -l darknet

In [ ]:
ls cfg

In [ ]:
cat cfg/yolov3.cfg

#学習済データを使って物体検出

yolov3を様々なデータで学習した成果(weightsファイル)が公開されているので、これをダウンロードしてきて、dataディレクトリ以下にある画像ファイル内の物体検知をしましょう。

学習済みデータをダウンロードします


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

ダウンロードしてきた学習済データを使って、`data/dog.jpg`に映っている物体の検出を行ないます

In [ ]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

結果は`predictinos.jpg`というファイルに出力されるので表示します

In [ ]:
from IPython.display import Image
display(Image('predictions.jpg'))

# 別のファイルでも試してみる

dataディレクトリの下にdog.jpg以外にも複数の画像ファイルが用意されているので
`dog.jpg`の部分を色々と置き換えて試してみましょう!
- data/dog.jpg
-    data/giraffe.jpg
-  data/kite.jpg
-    data/scream.jpg
- data/eagle.jpg 
- data/horses.jpg
-   data/person.jpg



In [ ]:
! ./darknet detect cfg/yolov3.cfg yolov3.weights  data/giraffe.jpg

In [ ]:
display(Image('predictions.jpg'))